<a href="https://colab.research.google.com/github/DiegoRomanCortes/AED/blob/pr%2F1/Tarea6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CC3001 Otoño 2020 Tarea 6

## Árboles 2-3

Nombre: Diego Román

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 20 de julio de 2020 a las 23:59

El objetivo de esta tarea es que usted implemente árboles 2-3. Si ve en el apunte, para los árboles AVL aparece la implementación, pero no así para los árboles 2-3. Usted debe ayudar a completar el apunte y si lo hace bien, es posible que en el futuro aparezca su código en este capítulo.

A continuación aparece algo de código para ayudarlo a partir. Hay dos tipos de nodos internos (``Nodo2`` y ``Nodo3``), según si ese nodo es binario o ternario, y un tipo de nodo externo u hoja (``Nodoe``). Además, está la clase ``Arbol23``, que es el punto de partida. Las operaciones que usted tiene que implementar son ``insert`` y ``search``.

En varias partes se ha rellenado con código provisorio, para que compile. Usted debe reemplazar y completar todo lo que sea necesario para que todo funcione.

El código esbozado más abajo está orientado a una implementación recursiva, la que podemos describir de la siguiente manera:

* Si se inserta una llave en un nodo binario, hay que insertarla recursivamente en el hijo izquierdo o en el derecho, según corresponda. Si esto no produce un *split* del hijo, el resultado es un puntero al nodo resultante (que puede ser binario o ternario), lo cual se anota en el lugar respectivo y se retorna un puntero al nodo resultante. Pero si el hijo se divide, se recibe el resultado que es una tupla que contiene el árbol izquierdo, el árbol derecho, y la llave que los separa. Con esto este nodo tiene la información para mutar a ternario y retornar el resultado.

* Si se inserta una llave en un nodo ternario hay que insertarla recursivamente en el hijo izquierdo, en el del medio o en el derecho, según corresponda. Si esto no produce un *split* del hijo, el resultado es un puntero al nodo resultante (que puede ser binario o ternario), lo cual se anota en el lugar respectivo y se retorna un puntero al nodo resultante. Pero si el hijo se divide, se recibe el resultado que es una tupla que contiene el árbol izquierdo, el árbol derecho, y la llave que los separa. Con esto este nodo tiene la información para a su vez dividirse y retornar la tupla resultante.

* Lo anterior permite modelar de la misma manera la inserción en una hoja: al recibir una llave, se divide en dos hojas, y la llave entrante las separa. Esto se retorna hacia arriba en la forma de una tupla y se la da el mismo tratamiento anterior.

* En la clase ``Arbol23`` hay que estar preparado para recibir una tupla como resultado de la inserción, lo cual indicaría que la raíz se dividió. En ese caso, hay que crear nodo binario para que sea la nueva raíz.

La operación de búsqueda (``search``) debe retornar un puntero al nodo que contiene la llave buscada, en caso de ser exitosa, o ``None`` si es infructuosa.

In [100]:
class Nodo2:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
        
    def insert(self,x):
        assert x != self.info #asegurarse que no haya elementos repetidos
        if x < self.info:
            p = self.izq.insert(x) #insertamos recursivamente donde corresponda
            if type(p) == tuple: #si es una tupla quiere decir que se produjo un split en la llamada recursiva
                (izq_new, info_new, der_new) = p
                return Nodo3(izq_new, info_new, der_new, self.info, self.der) #el nodo2 se convierte en nodo3 con los parámetros que recibe
            else: #caso sin split
                self.izq = p #se actualiza el hijo acorde a los cambios debidos a la insercion
                return self #se retorna el puntero
        else: #analogo
            p = self.der.insert(x)
            if type(p) == tuple:
                (izq_new, info_new, der_new) = p
                return Nodo3(self.izq, self.info, izq_new, info_new, der_new)
            else:
                self.der = p
                return self

    def search(self,x):
        if x == self.info: #encontramos x en el arbol
          return self #retornamos el puntero al nodo que lo contenía
        elif x < self.info: #debe estar en el hijo izquierdo
          return self.izq.search(x) #llamada recursiva
        else: #debe estar en el hijo derecho
          return self.der.search(x) #análogo
        
    def string(self):
        return ("("+self.izq.string()
                +str(self.info)
                +self.der.string()+")")

class Nodo3:
    def __init__(self, izq, info1, med, info2, der):
        self.izq=izq
        self.info1=info1
        self.med=med
        self.info2=info2
        self.der=der
        
    def insert(self,x):
        assert x != self.info1 and x != self.info2
        if x < self.info1:
          p = self.izq.insert(x)
          if type(p) == tuple:
                (izq_new, info_new, der_new) = p
                izq_new = Nodo2(izq_new, info_new, der_new)
                der_new = Nodo2(self.med, self.info2, self.der)
                return (izq_new, self.info1, der_new) #se devuelve una tuple para 'avisar' que hubo un split
          else:
                self.izq = p
                return self
        elif x > self.info1 and x < self.info2:
          p = self.med.insert(x)
          if type(p) == tuple:
                (izq_new, info_new, der_new) = p
                izq_new = Nodo2(self.izq, self.info1, izq_new)
                der_new = Nodo2(der_new, self.info2 ,self.der)
                return (izq_new, info_new, der_new)
          else:
                self.med = p
                return self
        else: #x > self.info2
          p = self.der.insert(x)
          if type(p) == tuple:
                (izq_new, info_new, der_new) = p
                der_new = Nodo2(izq_new, info_new, der_new)
                izq_new = Nodo2(self.izq, self.info1, self.med)
                return (izq_new, self.info2, der_new)
          else:
                self.der = p
                return self

    def search(self,x):
        #mismo razonamiento que para Nodo2, solo que hay más casos
        if x == self.info1 or x == self.info2:
          return self
        elif x < self.info1:
          return self.izq.search(x)
        elif x > self.info1 and x < self.info2:
          return self.med.search(x)
        else: #x > self.info2
          return self.der.search(x)

    def string(self):
        return ("("+self.izq.string()
                +str(self.info1)
                +self.med.string()
                +str(self.info2)
                +self.der.string()+")")    
    
class Nodoe:
    def __init__(self):
        pass

    def search(self, x):
        return None #Caso base crucial en las llamadas recursivas de Nodo2 y Nodo3

    def insert(self,x):
        #por defecto se produce un split de nodoe al tratar de insertar un elemento aquí
        #esto sirve de caso base para las llamadas recursivas
        return (Nodoe(), x, Nodoe()) 

    def string(self):
        return"☐"

class Arbol23:
    def __init__(self,raiz=Nodoe()):
        self.raiz=raiz     
        
    def insert(self,x):
        self.raiz = self.raiz.insert(x)
        #split de Nodo3
        if type(self.raiz) == tuple:
          (izq, info, der) = self.raiz
          self.raiz = Nodo2(izq, info, der)
        
    def search(self,x):
        return self.raiz.search(x)
    
    def imprimir(self):        
        print(self.raiz.string())

Para probar este código, vamos a construir "a mano" el árbol 2-3 que aparece en el apunte, y luego imprimirlo:

In [101]:
a=Nodo3(Nodo2(Nodoe(),10,Nodoe()),25,Nodo3(Nodoe(),32,Nodoe(),48,Nodoe()),57,Nodo2(Nodoe(),74,Nodoe()))

In [102]:
print(a.string())

((☐10☐)25(☐32☐48☐)57(☐74☐))


Pruebe su código comenzando con un árbol vacío e insertando la siguiente secuencia de llaves:

```
3, 1, 4, 5, 9, 6, 2
```

Imprima el árbol resultante después de cada inserción. A continuación, busque "2", busque "3" y busque "7".

In [103]:
arbol23 = Arbol23()

arbol23.insert(3)
arbol23.imprimir()

arbol23.insert(1)
arbol23.imprimir()

arbol23.insert(4)
arbol23.imprimir()

arbol23.insert(5)
arbol23.imprimir()

arbol23.insert(9)
arbol23.imprimir()

arbol23.insert(6)
arbol23.imprimir()

arbol23.insert(2)
arbol23.imprimir()

(☐3☐)
(☐1☐3☐)
((☐1☐)3(☐4☐))
((☐1☐)3(☐4☐5☐))
((☐1☐)3(☐4☐)5(☐9☐))
((☐1☐)3(☐4☐)5(☐6☐9☐))
((☐1☐2☐)3(☐4☐)5(☐6☐9☐))


In [104]:
dos = arbol23.search(2)
tres = arbol23.search(3)
siete = arbol23.search(7)

print(dos.string())
print(tres.string())
print(siete) #no está en el arbol23 por lo que debe ser None (no tiene método string())

(☐1☐2☐)
((☐1☐2☐)3(☐4☐)5(☐6☐9☐))
None


## ¿Qué hay que entregar?

Usted debe entregar este mismo archivo, modificado de acuerdo a lo que se pide. Haga todos los cambios necesarios para explicar y documentar adecuadamente su código. No olvide poner su nombre.